<a href="https://colab.research.google.com/github/brendanpshea/colab-utilities/blob/main/SQL_Quiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3

# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# Create a table (if it doesn't already exist)
cursor.execute('''
CREATE TABLE IF NOT EXISTS students (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL
)''')

# Insert sample data (this step is only necessary the first time)
cursor.execute('INSERT INTO students (name) VALUES ("Alice"), ("Bob"), ("Charlie")')

# Commit changes and close the connection
conn.commit()
conn.close()


In [11]:
import sqlite3
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from ipywidgets import Button, Layout, VBox, Textarea
import pandas as pd

# Function to display the table schemas in a nicely formatted HTML table
def show_table_schemas(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    schema_html = "<h2>Database Schema</h2>"
    schema_html += """
        <style>
            table {
                border-collapse: collapse; /* Makes the borders collapse into a single border */
                width: 100%; /* Optional: Sets the table width to full container width */
                margin-bottom: 20px; /* Adds some space after each table */
            }
            th, td {
                border: 1px solid black; /* Adds a solid black border around table headers and cells */
                padding: 8px; /* Adds some padding inside each cell for better readability */
                text-align: left; /* Aligns the text to the left. Change to 'center' if preferred */
            }

        </style>
        """

    for table in tables:
        schema_html += f"<h4>Table: {table[0]}</h4><br>"
        cursor.execute(f"PRAGMA table_info({table[0]})")
        columns = cursor.fetchall()
        schema_html += "<table><tr><th>Column Name</th><th>Data Type</th></tr>"
        for column in columns:
            schema_html += f"<tr><td>{column[1]}</td><td>{column[2]}</td></tr>"
        schema_html += "</table>"
    display(HTML(schema_html))
    conn.close()

# Main function to iterate through questions and process user inputs
def process_questions(db_path, questions, answers):
    if not questions or not answers or len(questions) != len(answers):
        display(HTML("<div>Please provide an equal number of questions and answers.</div>"))
        return

    question_index = 0  # To track the current question

    # Function to display the current question and set up UI for answer submission
    def display_current_question():
        clear_output(wait=True)
        show_table_schemas(db_path)
        question_html = f"<h3>SQL Question {question_index + 1}:</h3><p>{questions[question_index]}</p>"
        display(HTML(question_html))
        text_area.value = ''  # Clear previous input
        display(query_widget)

    # Function to handle query submission
    def submit_query(b):
        with sqlite3.connect(db_path) as conn:
            try:
                user_query = text_area.value
                user_result = pd.read_sql_query(user_query, conn)
                correct_query = answers[question_index]
                correct_result = pd.read_sql_query(correct_query, conn)

                # Display results
                display(HTML("<h4>Your Results:</h4>"))
                display(user_result)
                display(HTML("<h4>Expected Results:</h4>"))
                display(correct_result)

                # Check correctness
                if user_result.equals(correct_result):
                    feedback = "<div style='color: green;'><strong>Correct!</strong></div>"
                    next_button.layout.visibility = 'visible'  # Show the next button
                else:
                    feedback = "<div style='color: red;'><strong>Incorrect.</strong> Try again or proceed to the next question.</div>"
                    retry_button.layout.visibility = 'visible'  # Show the retry button
                display(HTML(feedback))
            except Exception as e:
                display(HTML(f"<div>Error executing your query: {str(e)}</div>"))

    # Function to proceed to the next question
    def next_question(b):
        nonlocal question_index
        question_index += 1
        if question_index < len(questions):
            display_current_question()
        else:
            display(HTML("<div>All questions completed. Well done!</div>"))
            next_button.layout.visibility = 'hidden'
            retry_button.layout.visibility = 'hidden'

    # Function to retry the current question
    def retry_question(b):
        display_current_question()

    text_area = Textarea(value='', placeholder='Type your SQL query here...', description='Query:', layout=Layout(width='60%', height='100px'))
    submit_button = Button(description="Submit")
    next_button = Button(description="Next Question", layout=Layout(visibility='hidden'))
    retry_button = Button(description="Retry", layout=Layout(visibility='hidden'))

    submit_button.on_click(submit_query)
    next_button.on_click(next_question)
    retry_button.on_click(retry_question)

    query_widget = VBox([text_area, submit_button, retry_button, next_button])

    display_current_question()  # Display the first question

# Example usage
db_path = 'example.db'
questions = [
    "Select all names from the students table.",
    "Select count(*) from students where name like 'A%';"
]

answers = [
    "SELECT name FROM students",
    "SELECT count(*) FROM students WHERE name LIKE 'A%'"
]

process_questions(db_path, questions, answers)


Column Name,Data Type
id,INTEGER
name,TEXT
